In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

from tqdm.notebook import tqdm
!pip install python_speech_features
from python_speech_features import mfcc, logfbank

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
drive_path = '/content/drive'
from google.colab import drive
drive.mount(drive_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Data/ERPdata.csv")


In [5]:
time_axis = df['time_ms'].unique()
sample_rate = (1/(time_axis[1]-time_axis[0]))*1000
del df

In [6]:
demographic = pd.read_csv("/content/drive/MyDrive/Data/demographic.csv")
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
column_list = pd.read_csv("/content/drive/MyDrive/Data/columnLabels.csv").columns
electrodes= list(pd.read_csv('/content/drive/MyDrive/Data/columnLabels.csv').columns[4:])

N_PERSON = 81
N_AVERAGED = 16
x_count = 0
num_electrodes = len(electrodes)
num_rows = 81 * 100
num_cols = 9216 * (num_electrodes // N_AVERAGED)
X = np.zeros((num_rows, num_cols), dtype=np.float32)
Y = np.zeros(len(X))

In [ ]:
def averaged_by_N_rows(a, n):
    assert a.ndim == 2, "Input matrix must be 2-dimensional"
    assert a.shape[0] % n == 0, "Number of rows in the input matrix must be divisible by n"
    return np.mean(a.reshape(-1, n, a.shape[1]), axis=1)

In [ ]:

for i in tqdm(range(N_PERSON)):

    person = str(i+1)
    csv_path = f"{'/content/drive/MyDrive/Data'}/{person}.csv/{person}.csv"
    df = pd.read_csv(csv_path, header=None, names=column_list)
    grouped_trials = df.groupby("trial")
    
    for trial_number, trial_data in grouped_trials:
        number_of_trials = len(trial_data)
        if number_of_trials == 9216:
            current_sample_matrix = trial_data[electrodes].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            np.append(X,averaged_by_N_big_vec.astype(np.float32))
            np.append(Y,diagnosis_dict[int(person)])

  0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
np.save('/content/drive/MyDrive/Data/X.npy', X);
np.save('/content/drive/MyDrive/Data/Y.npy', Y);

In [7]:
X = np.load('/content/drive/MyDrive/Data/X.npy')
Y = np.load('/content/drive/MyDrive/Data/Y.npy')

In [20]:
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)

X_norm_2d = (X_norm.reshape(X_norm.shape[0], len(electrodes), X_norm.shape[1] // len(electrodes), 1))


In [ ]:
X_mffc = np.zeros((X_norm_2d.shape[0],X_norm_2d.shape[1],56,13))
for i in tqdm(range(X_norm_2d.shape[0])):
  for j in (range(X_norm_2d.shape[1])):
    X_mffc[i][j] =mfcc(X_norm_2d[i][j],sample_rate,numcep=13,nfilt=26,nfft=1103)
np.save('/content/drive/MyDrive/Data/X_mffc.npy', X_mffc);

  0%|          | 0/7092 [00:00<?, ?it/s]